In [1]:
from dataclasses import dataclass
from functools import cached_property
import netCDF4
from datetime import datetime, timedelta
import numpy as np
from forcys.wavedata import WaveDataFile

In [166]:
@dataclass
class WindDataFile:
    """WindDataFile class"""
    dataset: netCDF4.Dataset

    @classmethod
    def from_file(cls, filename: str):
        dataset = netCDF4.Dataset(filename)
        return cls(dataset)

    @cached_property
    def variables(self) -> list[str]:
        variables = list(self.dataset.variables.keys())
        # drop_variables = ['time', 'latitude', 'longitude']
        # variables = [item for item in variables if item not in drop_variables]
        return variables

In [2]:
path = r'D:\mypython_projects\forcys\mass_downloads\wind\AgApostoloi'
filename = 'windAgApostoloi_2020.nc'
wdf = WaveDataFile.from_file(f'{path}/{filename}')

In [3]:
wdf.variables

['u100', 'v100', 'u10', 'v10']

In [5]:
wdf.datetime[:26]

masked_array(data=[datetime.datetime(2020, 1, 1, 0, 0),
                   datetime.datetime(2020, 1, 1, 1, 0),
                   datetime.datetime(2020, 1, 1, 2, 0),
                   datetime.datetime(2020, 1, 1, 3, 0),
                   datetime.datetime(2020, 1, 1, 4, 0),
                   datetime.datetime(2020, 1, 1, 5, 0),
                   datetime.datetime(2020, 1, 1, 6, 0),
                   datetime.datetime(2020, 1, 1, 7, 0),
                   datetime.datetime(2020, 1, 1, 8, 0),
                   datetime.datetime(2020, 1, 1, 9, 0),
                   datetime.datetime(2020, 1, 1, 10, 0),
                   datetime.datetime(2020, 1, 1, 11, 0),
                   datetime.datetime(2020, 1, 1, 12, 0),
                   datetime.datetime(2020, 1, 1, 13, 0),
                   datetime.datetime(2020, 1, 1, 14, 0),
                   datetime.datetime(2020, 1, 1, 15, 0),
                   datetime.datetime(2020, 1, 1, 16, 0),
                   datetime.datetime(2020

In [169]:
wdf.dataset.variables['time'][:20]

masked_array(data=[1051896, 1051897, 1051898, 1051899, 1051900, 1051901,
                   1051902, 1051903, 1051904, 1051905, 1051906, 1051907,
                   1051908, 1051909, 1051910, 1051911, 1051912, 1051913,
                   1051914, 1051915],
             mask=False,
       fill_value=999999)

In [181]:
base_date = datetime(1900, 1, 1)

t = int(wdf.dataset.variables['time'][0])
print(t)
time_delta = timedelta(hours=t)
target_date = base_date + time_delta

print(target_date.strftime('%Y-%m-%d %H:%M:%S'))

print(datetime.fromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S'))

1051896
2020-01-01 00:00:00
1970-01-13 06:11:36


In [183]:
datesnew = netCDF4.num2date(wdf.dataset.variables['time'][:],
                             wdf.dataset.variables['time'].units,
                            wdf.dataset.variables['time'].calendar)
datesnew[:20]

masked_array(data=[cftime.DatetimeGregorian(2020, 1, 1, 0, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 1, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 2, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 3, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 4, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 5, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 6, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 7, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 8, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 9, 0, 0, 0, has_year_zero=False),
                   cftime.DatetimeGregorian(2020, 1, 1, 10, 0, 0, 0, has_year_zero=False),
         

In [188]:
masked_array_datetime = np.ma.masked_array(
    data=[datetime(*d.timetuple()[:6]) for d in datesnew],
    mask=datesnew.mask
)
masked_array_datetime

masked_array(data=[datetime.datetime(2020, 1, 1, 0, 0),
                   datetime.datetime(2020, 1, 1, 1, 0),
                   datetime.datetime(2020, 1, 1, 2, 0), ...,
                   datetime.datetime(2020, 12, 31, 21, 0),
                   datetime.datetime(2020, 12, 31, 22, 0),
                   datetime.datetime(2020, 12, 31, 23, 0)],
             mask=False,
       fill_value='?',
            dtype=object)

In [171]:
# ttts = wdf.dataset.variables['time'][:]
# print(ttts[:])
# ttts2 = ttts + base_date
# print(ttts2[:])

In [179]:
timestamps = wdf.dataset.variables['time'][:]
datetimes = np.ma.array(np.datetime64('1900-01-01') + timestamps.astype('timedelta64[s]'), 
                     mask=timestamps.mask)
timedelta2 = np.timedelta64(time_delta, 's')
updated_datetimes = datetimes + timedelta2

updated_timestamps = np.ma.array((updated_datetimes - np.datetime64('1970-01-01')) / np.timedelta64(1, 's'), 
                              mask=timestamps.mask).astype(int)

print(datetime.fromtimestamp(updated_timestamps[0]))

2020-01-13 06:11:36
